In [38]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [39]:
import os
with open(os.path.join(os.curdir, 'datasets/gutenberg', 'book1.txt'), 'r') as fp:
    text = fp.read()

text = text[text.find('THE MYSTERIOUS ISLAND'): text.find('End of the Project Gutenberg EBook')]
#char_set = set(text)
#print('Text length: {}, Characters: {}'.format(len(text), len(char_set)))
print(len(text))

1114307


In [40]:
#chars_sorted = sorted(char_set)
#char2int = { ch:i for i, ch in enumerate(chars_sorted)}

#text_encoded = np.array([char2int[ch] for ch in text], dtype=np.int32)
#text_encoded.shape

In [41]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
text_encoded = np.array(tokenizer.texts_to_sequences(text)) - 1

In [42]:
text_ds = tf.data.Dataset.from_tensor_slices(text_encoded)

In [43]:
print(next(iter(text_ds)))

tf.Tensor([2], shape=(1,), dtype=int32)


In [8]:
#ds_chunks = text_ds.batch(41, drop_remainder=True)

#def split_input_target(chunk):
#    return chunk[:-1], chunk[1:]
#ds_sequences = ds_chunks.map(split_input_target)

In [44]:
ds_chunks = text_ds.window(41, shift=1, drop_remainder=True)
ds_chunks = ds_chunks.flat_map(lambda chunk: chunk.batch(41))
ds_sequences = ds_chunks.map(lambda chunk: (chunk[:-1], chunk[1:]))

In [45]:
print(next(iter(ds_sequences)))

(<tf.Tensor: shape=(40, 1), dtype=int32, numpy=
array([[ 2],
       [ 6],
       [ 1],
       [ 0],
       [18],
       [22],
       [ 9],
       [ 2],
       [ 1],
       [ 8],
       [ 7],
       [ 4],
       [14],
       [ 9],
       [ 0],
       [ 7],
       [ 9],
       [11],
       [ 3],
       [ 5],
       [10],
       [13],
       [13],
       [21],
       [22],
       [ 0],
       [34],
       [14],
       [11],
       [ 1],
       [ 9],
       [ 0],
       [24],
       [ 1],
       [ 8],
       [ 5],
       [ 1],
       [13],
       [13],
       [38]])>, <tf.Tensor: shape=(40, 1), dtype=int32, numpy=
array([[ 6],
       [ 1],
       [ 0],
       [18],
       [22],
       [ 9],
       [ 2],
       [ 1],
       [ 8],
       [ 7],
       [ 4],
       [14],
       [ 9],
       [ 0],
       [ 7],
       [ 9],
       [11],
       [ 3],
       [ 5],
       [10],
       [13],
       [13],
       [21],
       [22],
       [ 0],
       [34],
       [14],
       [11],
       [ 1],
     

In [46]:
ds_sequences = ds_sequences.map(lambda example, prediction: (tf.squeeze(example, [1]), tf.squeeze(prediction, [1]) ))
train_ds = ds_sequences.shuffle(10000).batch(32)

In [48]:
print(next(iter(ds_sequences)))
print(len(tokenizer.word_index))

(<tf.Tensor: shape=(40,), dtype=int32, numpy=
array([ 2,  6,  1,  0, 18, 22,  9,  2,  1,  8,  7,  4, 14,  9,  0,  7,  9,
       11,  3,  5, 10, 13, 13, 21, 22,  0, 34, 14, 11,  1,  9,  0, 24,  1,
        8,  5,  1, 13, 13, 38])>, <tf.Tensor: shape=(40,), dtype=int32, numpy=
array([ 6,  1,  0, 18, 22,  9,  2,  1,  8,  7,  4, 14,  9,  0,  7,  9, 11,
        3,  5, 10, 13, 13, 21, 22,  0, 34, 14, 11,  1,  9,  0, 24,  1,  8,
        5,  1, 13, 13, 38, 45])>)
55


In [49]:
network = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index), output_dim=128),
    tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.3),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(tokenizer.word_index)))
])
network.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
network.fit(train_ds, epochs=15)

Epoch 1/15
   7414/Unknown - 1596s 215ms/step - loss: 1.6862 - accuracy: 0.5013

KeyboardInterrupt: 

In [50]:
char_array = np.array(chars_sorted)
def sample(model, starting_str, len_generated_text = 200):
    encoded_input = [char2int[ch] for ch in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str
    #print(generated_str)

    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)
        #print(logits)

        scaled_logits = logits * 1.0

        new_charindx = tf.random.categorical(scaled_logits, num_samples=1)
        #print(new_charindx)

        new_char_indx = tf.squeeze(new_charindx)[-1].numpy()
        #print(char_array[new_char_indx])
        generated_str += str(char_array[new_char_indx])

        new_char_indx = tf.expand_dims([new_char_indx], 0)
        encoded_input = tf.concat([encoded_input, new_char_indx], axis=1)
        encoded_input = encoded_input[:, -40:]

    return generated_str

In [51]:
tf.random.set_seed(42)
print(sample(network, starting_str='The island'))

InvalidArgumentError: indices[0,1] = 57 is not in [0, 55) [Op:ResourceGather]